# The Ultimate Customer Segmentation Analysis

In [1]:
# Importing the required libraries
import pandas as pd
import numpy as np
import math
from datetime import datetime

### Loading the Data

In [2]:
# file path
file_path = "mock_data.csv"

# sum number of rows in the dataset
total_rows = sum(1 for i in open(file_path))-1


# first 75% of the rows
rows_to_df = math.floor(total_rows * 0.75)


print(f"Total rows: {total_rows}")
print(f"75% of rows: {rows_to_df}")

Total rows: 1000
75% of rows: 750


In [3]:
# Load first 75% of the dataset
df = pd.read_csv(file_path, nrows=rows_to_df)
df.shape

(750, 4)

In [4]:
# checking the data format of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Customer ID      750 non-null    int64 
 1   Country          750 non-null    object
 2   Total Purchases  750 non-null    int64 
 3   Sign-up Date     750 non-null    object
dtypes: int64(2), object(2)
memory usage: 23.6+ KB


### Data Cleaning

In [5]:
# Remove duplicates that appear in consecutive order
df_cleaned = df.loc[(df != df.shift()).any(axis=1)].copy().reset_index(drop = True)
df_cleaned.shape

(750, 4)

In [6]:
# Converting 'Sign-up Date' column to proper datetime format
df_cleaned['Sign-up Date']=pd.to_datetime(df_cleaned['Sign-up Date'])
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Customer ID      750 non-null    int64         
 1   Country          750 non-null    object        
 2   Total Purchases  750 non-null    int64         
 3   Sign-up Date     750 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 23.6+ KB


In [7]:
df_cleaned.head(10)

,Customer ID,Country,Total Purchases,Sign-up Date
0,1015,Palestinian Territory,12,2022-04-03 21:42:29
1,1029,China,47,2016-01-08 03:13:12
2,1025,Nigeria,10,2023-02-20 19:06:14
3,1006,United Kingdom,48,2022-05-26 12:46:35
4,1024,Philippines,16,2016-12-13 03:44:04
5,1033,Indonesia,20,2016-05-26 22:29:44
6,1017,Paraguay,14,2022-09-08 16:30:56
7,1035,Honduras,15,2021-04-14 18:55:03
8,1029,Argentina,14,2020-11-01 10:21:03
9,1014,China,3,2020-04-14 00:51:05


In [8]:
# Replacing all entries from year 2020 with NaN
df_cleaned['Sign-up Date'] = df_cleaned['Sign-up Date'].mask(df_cleaned['Sign-up Date'].dt.year == 2020, np.nan)
df_cleaned.head(10)

,Customer ID,Country,Total Purchases,Sign-up Date
0,1015,Palestinian Territory,12,2022-04-03 21:42:29
1,1029,China,47,2016-01-08 03:13:12
2,1025,Nigeria,10,2023-02-20 19:06:14
3,1006,United Kingdom,48,2022-05-26 12:46:35
4,1024,Philippines,16,2016-12-13 03:44:04
5,1033,Indonesia,20,2016-05-26 22:29:44
6,1017,Paraguay,14,2022-09-08 16:30:56
7,1035,Honduras,15,2021-04-14 18:55:03
8,1029,Argentina,14,NaT
9,1014,China,3,NaT


In [9]:
# Dropping all rows total purchase is less than 5 and customer is from Canada
df_cleaned = df_cleaned[~((df_cleaned['Country'] == 'Canada') & (df_cleaned['Total Purchases'] < 5))]
df_cleaned.shape

(748, 4)

### Feature Engineering

In [10]:
# Adding a new column called Loyalty score
df_cleaned['membership_years'] = (pd.Timestamp(datetime.now()) - df_cleaned['Sign-up Date']).dt.days / 365


# Define a function for loyalty score
def Loyalty_Score (row):
    if row['Total Purchases'] > 20:
        return 3
    elif row['Total Purchases'] <= 10 <= row['Total Purchases'] <=20 :
        return 2
    else:
        if row['membership_years'] > 3 :
            return 2
        else:
            return 1

df_cleaned['Loyalty Score'] = df_cleaned.apply(Loyalty_Score, axis=1)
df_cleaned.head(3)
    

,Customer ID,Country,Total Purchases,Sign-up Date,membership_years,Loyalty Score
0,1015,Palestinian Territory,12,2022-04-03 21:42:29,2.961644,1
1,1029,China,47,2016-01-08 03:13:12,9.202740,3
2,1025,Nigeria,10,2023-02-20 19:06:14,2.076712,2


### Data Agrgregaton and Filtering

In [16]:
# Grouping Total Purchases by Country (except Mexico) and calculating the mean and maximum purchase
df_filtered = df_cleaned[df_cleaned['Country'] != 'Mexico']
total_purchases_stats = df_filtered.groupby('Country')['Total Purchases'].agg(['mean', 'max']).reset_index()
total_purchases_stats

,Country,mean,max
0,Afghanistan,8.000000,20
1,Albania,20.500000,38
2,Andorra,11.000000,11
3,Angola,15.000000,29
4,Argentina,24.692308,45
...,...,...,...
106,Vanuatu,22.000000,22
107,Venezuela,25.333333,43
108,Vietnam,24.333333,47
109,Yemen,10.000000,15


In [17]:
# Most Loyalty Score for each Country excluding Mexico
most_common_loyalty_score_by_country = df_filtered.groupby('Country')['Loyalty Score'].agg(lambda x: x.mode().iloc[0])
most_common_loyalty_score_by_country

Country
Afghanistan    2
Albania        2
Andorra        2
Angola         2
Argentina      3
              ..
Vanuatu        3
Venezuela      3
Vietnam        3
Yemen          2
Zimbabwe       2
Name: Loyalty Score, Length: 111, dtype: int64

### Data Export and Report Generation

In [24]:
# Save the process data with the desired columns
final_columns = ["Customer ID", "Loyalty Score", "Country", "Total Purchases"]
processed_df = df_cleaned[final_columns ]
processed_df.head(2)

,Customer ID,Loyalty Score,Country,Total Purchases
0,1015,1,Palestinian Territory,12
1,1029,3,China,47


In [26]:
# Getting today's date
today = datetime.today()
date_str = today.strftime("%d_%m_%Y")

# Save the df to CSV file
filename = f"processed_data_{date_str}.csv"
processed_df.to_csv(filename, index=False)

filename

'processed_data_20_03_2025.csv'

In [35]:
# printing the first 10 rows of the processed df, using random state for reproducibilty
shuffled_df = processed_df.sample(frac= 1, random_state=4)
shuffled_df.head(10)

,Customer ID,Loyalty Score,Country,Total Purchases
741,1025,2,Russia,3
560,1023,1,China,18
33,1033,2,Azerbaijan,1
274,1023,2,Indonesia,12
639,1011,3,Sweden,35
209,1024,3,Bulgaria,24
67,1013,3,China,47
646,1017,3,China,28
157,1029,3,China,24
552,1033,3,Colombia,45


### Summary of Findings from the Analysis
- The mock dataset used for this analysis originally contained 1000 rows and 4 columns, however, the first 750 rows was loaded in the the pd dataframe for the purpose of the analysis.
- For the data cleaning steps, consecutive duplicates were removed and the data formats for each column was checked to ensure they are in the right or standard formats
- New features(Membership years and Loyalty scores) were created out of the existing features. This is to provide insights that will be useful in developing targeted marketing strategies and customer retention programs that will propel the business forward.
- Data aggregation and filtering was done to get the total purchases by country and search out most common loyalty score by country.Some countries demonstrated high purchase values, suggesting regions with potential for targeted marketing strategies
- The final proccessed data was saved with the desired column in a downloadable csv format.